In [2]:
import os
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

from torch import nn, optim
from torchvision import transforms, datasets, models
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
# Data Preprocessing

data_dir = "dataset"

transform = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ]),
    "test": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
}

train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=transform["train"])
test_dataset = datasets.ImageFolder(os.path.join(data_dir, "test"), transform=transform["test"])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

class_names = train_dataset.classes
print(f"Classes: {class_names}")


Classes: ['benign', 'malignant']


In [5]:
# Load model with pretrained weights using new API
weights = ResNet18_Weights.DEFAULT
model = resnet18(weights=weights)

# Freeze base layers
for param in model.parameters():
    param.requires_grad = False

# Modify the classifier
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 128),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(128, len(class_names)),
)

model = model.to(device)


In [6]:
# Loss and Optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)


In [7]:
# Training with tqdm Progress Bar

def train_model(model, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            progress_bar.set_postfix(loss=running_loss / (progress_bar.n + 1))

    print("Training complete.")
    torch.save(model.state_dict(), "model.pth")

train_model(model, criterion, optimizer, num_epochs=20)


Epoch 1/10:   0%|          | 0/301 [00:00<?, ?it/s]

Epoch 2/10:   0%|          | 0/301 [00:00<?, ?it/s]

Epoch 3/10:   0%|          | 0/301 [00:00<?, ?it/s]

Epoch 4/10:   0%|          | 0/301 [00:00<?, ?it/s]

Epoch 5/10:   0%|          | 0/301 [00:00<?, ?it/s]

...

Epoch 7/10:   0%|          | 0/301 [00:00<?, ?it/s]

Epoch 8/10:   0%|          | 0/301 [00:00<?, ?it/s]

Epoch 9/10:   0%|          | 0/301 [00:00<?, ?it/s]

Epoch 10/10:   0%|          | 0/301 [00:00<?, ?it/s]

Training complete.


In [8]:
torch.save(model.state_dict(), "model.pth")

In [9]:
# Evaluation with tqdm

def evaluate_model(model):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        progress_bar = tqdm(test_loader, desc="Evaluating")
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            progress_bar.set_postfix(accuracy=f"{100 * correct / total:.2f}%")

    print(f"Final Test Accuracy: {100 * correct / total:.2f}%")

evaluate_model(model)


Evaluating:   0%|          | 0/32 [00:00<?, ?it/s]

Final Test Accuracy: 91.40%
